# **`pyjet` package demo**

This demo assumes that you have run `python -m pip install pyjet`. The demo uses Python 3, though the package also supports Python 2 for now.

## Example/demo event sample

The package contains a data sample for example purposes. The file lies under the `examples/` directory.
It is the sample (single-)event shipped with FastJet, with E moved to the first column.
The event format is `dtype=np.dtype([('E', 'f8'), ('px', 'f8'), ('py', 'f8'), ('pz', 'f8')])`.

All particles - hereby called the *event* - are handily retrieved as follows:

In [1]:
from pyjet.testdata import get_event

event = get_event()

print('Event with {} particles'.format(len(event)))

Event with 354 particles


## Basic usage

Perform the clustering, i.e. jet finding: `pyjet` provides as main API the `cluster()` function to make it easy to run the jet finder with reasonable default settings (see later for how to explicitly managed everything via dedicated classes).

In [2]:
from pyjet import cluster

sequence = cluster(event, R=0.6, p=-1)

jets = sequence.inclusive_jets()  # list of PseudoJets

print('# of jets: {}'.format(len(jets)))

# of jets: 91


Print basic information as a demonstration of functionality:

In [3]:
print("{0: <5} {1: >10} {2: >10} {3: >10} {4: >10} {5: >10}".format(
    "jet#", "pT", "eta", "phi", "mass", "#constit."))
for i, jet in enumerate(jets[:6]):
    print("{0: <5} {1: 10.3f} {2: 10.3f} {3: 10.3f} {4: 10.3f} {5: 10}".format(
        i + 1, jet.pt, jet.eta, jet.phi, jet.mass, len(jet)))

jet#          pT        eta        phi       mass  #constit.
1        983.280     -0.868      2.905     36.457         34
2        901.745      0.221     -0.252     51.850         34
3         67.994     -1.194     -0.200     11.984         32
4         12.465      0.433      0.673      5.461         13
5          6.568     -2.629      1.133      2.099          9
6          6.498     -1.828     -2.248      3.309          6


The 6th jet has the following constituents:

In [4]:
for constit in jets[5]:
    print(constit)

PseudoJet(pt=0.096, eta=-2.166, phi=-2.271, mass=0.000)
PseudoJet(pt=2.200, eta=-1.747, phi=-1.972, mass=0.140)
PseudoJet(pt=1.713, eta=-2.037, phi=-2.469, mass=0.940)
PseudoJet(pt=0.263, eta=-1.682, phi=-2.564, mass=0.140)
PseudoJet(pt=1.478, eta=-1.738, phi=-2.343, mass=0.940)
PseudoJet(pt=0.894, eta=-1.527, phi=-2.250, mass=0.140)


Get the constituents as an array (pT, eta, phi, mass),

In [5]:
print(jets[5].constituents_array())

[(0.09551261, -2.16560157, -2.27109083, 4.8909123e-06)
 (2.19975694, -1.74672746, -1.97178728, 1.3957000e-01)
 (1.71301882, -2.03656511, -2.46861524, 9.3957000e-01)
 (0.26339374, -1.68243005, -2.56397904, 1.3957000e-01)
 (1.47781519, -1.7378898 , -2.34304346, 9.3957000e-01)
 (0.89353864, -1.52729244, -2.24973202, 1.3957000e-01)]


... or as an array (E, px, py, pz),

In [6]:
print(jets[5].constituents_array(ep=True))

[(0.42190436, -0.06155242, -0.07303395, -0.41095089)
 (6.50193926, -0.85863306, -2.02526044, -6.11692764)
 (6.74203628, -1.33952806, -1.06775374, -6.45273802)
 (0.74600384, -0.22066287, -0.1438199 , -0.68386087)
 (4.43164941, -1.0311407 , -1.05862485, -4.07096881)
 (2.15920027, -0.56111108, -0.69538886, -1.96067711)]


## Jet substructure with exclusive jets

Reclustering the constituents of the hardest jet with the kt algorithm:

In [7]:
cs2 = cluster(jets[0].constituents_array(), R=0.6, p=1)
print(cs2.inclusive_jets())

[PseudoJet(pt=983.280, eta=-0.868, phi=2.905, mass=36.457)]


Go back in the clustering sequence to when there were two jets:

In [8]:
for subj in cs2.exclusive_jets(2):
    print(subj)

PseudoJet(pt=946.493, eta=-0.870, phi=2.908, mass=20.117)
PseudoJet(pt=36.921, eta=-0.800, phi=2.821, mass=4.119)


Ask how many jets there are with a given dcut:

In [9]:
dcut = 0.5

njets = cs2.n_exclusive_jets(dcut)
print("There are {0} jets with a dcut of {1}".format(njets, dcut))

There are 9 jets with a dcut of 0.5


Get the jets with the given dcut:

In [10]:
ejets_dcut = cs2.exclusive_jets_dcut(dcut)

for i, jet in enumerate(ejets_dcut):
    print(i + 1, jet)

1 PseudoJet(pt=308.478, eta=-0.865, phi=2.908, mass=2.119)
2 PseudoJet(pt=256.731, eta=-0.868, phi=2.906, mass=0.140)
3 PseudoJet(pt=142.326, eta=-0.886, phi=2.912, mass=0.829)
4 PseudoJet(pt=135.971, eta=-0.870, phi=2.910, mass=0.140)
5 PseudoJet(pt=91.084, eta=-0.864, phi=2.899, mass=1.530)
6 PseudoJet(pt=30.970, eta=-0.831, phi=2.822, mass=2.124)
7 PseudoJet(pt=7.123, eta=-0.954, phi=2.939, mass=1.017)
8 PseudoJet(pt=5.951, eta=-0.626, phi=2.818, mass=0.748)
9 PseudoJet(pt=4.829, eta=-0.812, phi=3.037, mass=0.384)


## Basic package classes

In [11]:
from pyjet import PseudoJet, JetDefinition, ClusterSequence, ClusterSequenceArea

Note that these classes are not automatically available via a `from pyjet import *`.
As such, much of their functionality available in FastJet isn't exposed on the Python side for now.

### `PseudoJet` class

In [12]:
jets[0]

PseudoJet(pt=983.280, eta=-0.868, phi=2.905, mass=36.457)

In [13]:
len(jets[0].constituents())

34

### `JetDefinition` class
- `algo`: name of the jet algorithm to use
- `R`: jet finding resolution parameter

In [14]:
algorithm = 'antikt'
R = 0.5

jet_def = JetDefinition(algo = algorithm, R = R)
jet_def

## Clustering in full control

The `cluster()` function wraps all steps for convenience but it is nevertheless possible to specify and run all steps explicitly.

Create an event with 3 particles:

In [15]:
import numpy as np
from pyjet import DTYPE_EP

particles = np.array([(100., 99., 0.1, 0.), (5., 4., -0.1, 0.), (99., -99.0, 0., 0.)], dtype=DTYPE_EP)
particles

array([(100.,  99.,  0.1, 0.), (  5.,   4., -0.1, 0.),
       ( 99., -99.,  0. , 0.)],
      dtype=[('E', '<f8'), ('px', '<f8'), ('py', '<f8'), ('pz', '<f8')])

Choose a jet definition:

In [16]:
algorithm = 'antikt'
R = 0.7

jet_def = JetDefinition(algo = algorithm, R = R)
jet_def

Run the clustering:

In [17]:
cs = ClusterSequence(particles, jet_def, ep=True)
cs

Extract the jets:

In [18]:
jets = cs.inclusive_jets()
jets

[PseudoJet(pt=103.000, eta=0.000, phi=0.000, mass=20.396),
 PseudoJet(pt=99.000, eta=0.000, phi=3.142, mass=0.000)]

Print the jets:

In [19]:
print("          pt eta phi")
for i in range(len(jets)):
    print("jet {i}: {pt} {eta} {phi}".format(i=i, pt=jets[i].pt, eta=jets[i].eta, phi=jets[i].phi))
    constituents = jets[i].constituents()
    for j in range(len(constituents)):
      print("    constituent {j}'s pt: {pt}".format(j=j, pt=constituents[j].pt))

          pt eta phi
jet 0: 103.0 0.0 0.0
    constituent 0's pt: 99.00005050503762
    constituent 1's pt: 4.001249804748512
jet 1: 99.0 0.0 3.141592653589793
    constituent 0's pt: 99.0


## Dress `PseudoJets` with additional information

You can associate arbitrary additional information to each particle and this information can be accessed as attributes of the PseudoJets.

In [20]:
import numpy as np
from numpy.lib.recfunctions import append_fields
from numpy.testing import assert_array_equal

event = get_event()
event = append_fields(event, 'id', data=np.arange(len(event)))

sequence = cluster(event, R=0.6, p=-1)
jets = sequence.inclusive_jets()

In [21]:
ids = []
for jet in jets:
    for constit in jet:
        ids.append(constit.id)
ids.extend([p.id for p in sequence.unclustered_particles()])

Are all particles accounted for?

In [22]:
assert_array_equal(sorted(ids), np.arange(len(event)))